In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Kidney_Papillary_Cell_Carcinoma"
cohort = "GSE42977"

# Input paths
in_trait_dir = "../../input/GEO/Kidney_Papillary_Cell_Carcinoma"
in_cohort_dir = "../../input/GEO/Kidney_Papillary_Cell_Carcinoma/GSE42977"

# Output paths
out_data_file = "../../output/preprocess/Kidney_Papillary_Cell_Carcinoma/GSE42977.csv"
out_gene_data_file = "../../output/preprocess/Kidney_Papillary_Cell_Carcinoma/gene_data/GSE42977.csv"
out_clinical_data_file = "../../output/preprocess/Kidney_Papillary_Cell_Carcinoma/clinical_data/GSE42977.csv"
json_path = "../../output/preprocess/Kidney_Papillary_Cell_Carcinoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Sequential Binary Gene-Ratio Tests Define a Novel Molecular Diagnostic Strategy for Malignant Pleural Mesothelioma"
!Series_summary	"The gene-expression ratio  technique was used to design a molecular signature to diagnose MPM from among other potentially confounding diagnoses and differentiate the epithelioid from the sarcomatoid histological subtype of MPM."
!Series_overall_design	"Microarray analysis was performed on 113 specimens including MPMs and a spectrum of tumors and benign tissues comprising the differential diagnosis of MPM.  A sequential combination of binary gene-expression ratio tests was developed to discriminate MPM from other thoracic malignancies .  This method was compared to other bioinformatic tools and this signature was validated  in an independent set of 170 samples.  Functional enrichment analysis was performed to identify differentially expressed probes."
Sample Characteristics Dictionary:
{0: ['tissue: control', 'tissue

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import numpy as np
import os
import json
from typing import Optional, Callable, Dict, Any, List, Union

# Let's analyze the sample characteristics dictionary to determine data availability
sample_chars = {0: ['tissue: control', 'tissue: Spindle Cell Sarcoma', 'tissue: Sarcoma', 'tissue: Metastatic Melanoma', 'tissue: Pleomorphic Sarcoma', 'tissue: Renal Cell Carcinoma-Clear Cell', 'tissue: Synovial Sarcoma', 'tissue: Metastatic Thymoma', 'tissue: Metastatic Prostate Cancer', 'tissue: Stomach Cancer-Stromal Sarcoma', 'tissue: Non-Hodgkins Lymphoma', 'tissue: Hemangioendothelioma', 'tissue: Papillary Thyroid Carcinoma', 'tissue: Metastatic Thyroid Cancer', 'tissue: Lymphocytic Lymphoma', 'tissue: Thymoma', 'tissue: Melanoma-Malignant', 'tissue: Hemangiopericytoma', 'tissue: Thyroid Carcinoma', 'tissue: Monophasic Synovial Sarcoma', 'tissue: Metastatic Alveolar Soft Part Sarcoma', 'tissue: Metastatic Meningeal Hemangiopericytoma', 'tissue: Follicular Lymphoma', 'tissue: Rhabdomyosarcoma', 'tissue: Myofibrosarcoma', 'tissue: Renal Cell Carcinoma - Chromophobe', 'tissue: MPM Epithelioid', 'tissue: MPM Sarcomatoid', 'tissue: MPM Biphasic', 'tissue: Normal Lung']}

# 1. Gene Expression Data Availability
# Based on the background information, it appears that this dataset contains microarray analysis,
# which suggests gene expression data is likely available
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Trait (Kidney Papillary Cell Carcinoma)
# From the sample characteristic dictionary, we can see that row 0 contains 'tissue' information
trait_row = 0

# Check for Kidney Papillary Cell Carcinoma in the data
renal_samples = [val for val in sample_chars[0] if "Renal" in val or "Kidney" in val]
papillary_samples = [val for val in sample_chars[0] if "Papillary" in val]

# We find "Renal Cell Carcinoma-Clear Cell" and "Renal Cell Carcinoma - Chromophobe"
# but not specifically Kidney Papillary Cell Carcinoma
# Looking at the data, it seems this study is about Malignant Pleural Mesothelioma (MPM)
# The trait we are looking for (Kidney_Papillary_Cell_Carcinoma) is not present
is_trait_available = False

# 2.2 Age
# There doesn't appear to be age information in the sample characteristics
age_row = None

# 2.3 Gender
# There doesn't appear to be gender information in the sample characteristics
gender_row = None

# Function to convert trait values - this won't be used since trait data is not available
# but we define it for completeness
def convert_trait(value):
    if value is None:
        return None
    
    # Extract the part after colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Since we're looking for Kidney Papillary Cell Carcinoma but the data is about MPM,
    # we have to mark trait as not available
    return None

# Define convert_age and convert_gender as placeholders
def convert_age(value):
    return None

def convert_gender(value):
    return None

# 3. Save Metadata
# Save metadata using the validate_and_save_cohort_info function
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction (if trait_row is not None and is_trait_available is True)
if trait_row is not None and is_trait_available:
    # Since we determined that this dataset doesn't contain our trait of interest,
    # we won't need to extract clinical features
    # This block won't execute due to is_trait_available being False
    pass
else:
    print("No relevant clinical data available for extraction for the specified trait.")


No relevant clinical data available for extraction for the specified trait.


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Add diagnostic code to check file content and structure
print("Examining matrix file structure...")
with gzip.open(matrix_file, 'rt') as file:
    table_marker_found = False
    lines_read = 0
    for i, line in enumerate(file):
        lines_read += 1
        if '!series_matrix_table_begin' in line:
            table_marker_found = True
            print(f"Found table marker at line {i}")
            # Read a few lines after the marker to check data structure
            next_lines = [next(file, "").strip() for _ in range(5)]
            print("First few lines after marker:")
            for next_line in next_lines:
                print(next_line)
            break
        if i < 10:  # Print first few lines to see file structure
            print(f"Line {i}: {line.strip()}")
        if i > 100:  # Don't read the entire file
            break
    
    if not table_marker_found:
        print("Table marker '!series_matrix_table_begin' not found in first 100 lines")
    print(f"Total lines examined: {lines_read}")

# 2. Try extracting gene expression data from the matrix file again with better diagnostics
try:
    print("\nAttempting to extract gene data from matrix file...")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {str(e)}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")

# If data extraction failed, try an alternative approach using pandas directly
if not is_gene_available:
    print("\nTrying alternative approach to read gene expression data...")
    try:
        with gzip.open(matrix_file, 'rt') as file:
            # Skip lines until we find the marker
            for line in file:
                if '!series_matrix_table_begin' in line:
                    break
            
            # Try to read the data directly with pandas
            gene_data = pd.read_csv(file, sep='\t', index_col=0)
            
            if not gene_data.empty:
                print(f"Successfully extracted gene data with alternative method: {gene_data.shape}")
                print("First 20 gene IDs:")
                print(gene_data.index[:20])
                is_gene_available = True
            else:
                print("Alternative extraction method also produced empty data")
    except Exception as e:
        print(f"Alternative extraction failed: {str(e)}")


Examining matrix file structure...
Line 0: !Series_title	"Sequential Binary Gene-Ratio Tests Define a Novel Molecular Diagnostic Strategy for Malignant Pleural Mesothelioma"
Line 1: !Series_geo_accession	"GSE42977"
Line 2: !Series_status	"Public on May 01 2013"
Line 3: !Series_submission_date	"Dec 18 2012"
Line 4: !Series_last_update_date	"Feb 16 2021"
Line 5: !Series_pubmed_id	"23493352"
Line 6: !Series_pubmed_id	"33540554"
Line 7: !Series_summary	"The gene-expression ratio  technique was used to design a molecular signature to diagnose MPM from among other potentially confounding diagnoses and differentiate the epithelioid from the sarcomatoid histological subtype of MPM."
Line 8: !Series_overall_design	"Microarray analysis was performed on 113 specimens including MPMs and a spectrum of tumors and benign tissues comprising the differential diagnosis of MPM.  A sequential combination of binary gene-expression ratio tests was developed to discriminate MPM from other thoracic malignanci

Successfully extracted gene data with 46713 rows
First 20 gene IDs:
Index(['ILMN_10000', 'ILMN_100000', 'ILMN_100007', 'ILMN_100009', 'ILMN_10001',
       'ILMN_100010', 'ILMN_10002', 'ILMN_100028', 'ILMN_100030',
       'ILMN_100031', 'ILMN_100034', 'ILMN_100037', 'ILMN_10004', 'ILMN_10005',
       'ILMN_100054', 'ILMN_100059', 'ILMN_10006', 'ILMN_100075',
       'ILMN_100079', 'ILMN_100083'],
      dtype='object', name='ID')

Gene expression data available: True


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers in the gene expression data
# The identifiers start with "ILMN_" which indicates they are Illumina probe IDs
# These are not standard human gene symbols and need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Extract gene annotation data from the SOFT file
print("Extracting gene annotation data from SOFT file...")
try:
    # Use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation data with {len(gene_annotation.index)} rows")
    
    # Preview the annotation DataFrame
    print("\nGene annotation preview (first few rows):")
    print(preview_df(gene_annotation))
    
    # Show column names to help identify which columns we need for mapping
    print("\nColumn names in gene annotation data:")
    print(gene_annotation.columns.tolist())
    
    # Check for relevant mapping columns
    if 'GB_ACC' in gene_annotation.columns:
        print("\nThe dataset contains GenBank accessions (GB_ACC) that could be used for gene mapping.")
        # Count non-null values in GB_ACC column
        non_null_count = gene_annotation['GB_ACC'].count()
        print(f"Number of rows with GenBank accessions: {non_null_count} out of {len(gene_annotation)}")
    
    if 'SPOT_ID' in gene_annotation.columns:
        print("\nThe dataset contains genomic regions (SPOT_ID) that could be used for location-based gene mapping.")
        print("Example SPOT_ID format:", gene_annotation['SPOT_ID'].iloc[0])
    
except Exception as e:
    print(f"Error processing gene annotation data: {e}")
    is_gene_available = False


Extracting gene annotation data from SOFT file...


Successfully extracted gene annotation data with 5512256 rows

Gene annotation preview (first few rows):
{'ID': ['ILMN_89282', 'ILMN_35826', 'ILMN_25544', 'ILMN_132331', 'ILMN_105017'], 'GB_ACC': ['BU678343', 'XM_497527.2', 'NM_018433.3', 'AW629334', 'AI818233'], 'Symbol': [nan, 'LOC441782', 'JMJD1A', nan, nan], 'SEQUENCE': ['CTCTCTAAAGGGACAACAGAGTGGACAGTCAAGGAACTCCACATATTCAT', 'GGGGTCAAGCCCAGGTGAAATGTGGATTGGAAAAGTGCTTCCCTTGCCCC', 'CCAGGCTGTAAAAGCAAAACCTCGTATCAGCTCTGGAACAATACCTGCAG', 'CCAGACAGGAAGCATCAAGCCCTTCAGGAAAGAATATGCGAGAGTGCTGC', 'TGTGCAGAAAGCTGATGGAAGGGAGAAAGAATGGAAGTGGGTCACACAGC'], 'Definition': ['UI-CF-EC0-abi-c-12-0-UI.s1 UI-CF-EC0 Homo sapiens cDNA clone UI-CF-EC0-abi-c-12-0-UI 3, mRNA sequence', 'PREDICTED: Homo sapiens similar to spectrin domain with coiled-coils 1 (LOC441782), mRNA.', 'Homo sapiens jumonji domain containing 1A (JMJD1A), mRNA.', 'hi56g05.x1 Soares_NFL_T_GBC_S1 Homo sapiens cDNA clone IMAGE:2976344 3, mRNA sequence', 'wk77d04.x1 NCI_CGAP_Pan1 Homo sapiens 

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the relevant columns for gene mapping
# From the gene annotation preview, we can see:
# - 'ID' column contains probe IDs with 'ILMN_' prefix (matching our gene expression data)
# - 'Symbol' column contains the gene symbols we need

print("Creating gene mapping dataframe...")
try:
    # Extract the mapping data from gene annotation
    mapping_df = get_gene_mapping(gene_annotation, 'ID', 'Symbol')
    print(f"Created mapping dataframe with {len(mapping_df)} entries")
    
    # Preview the mapping data
    print("\nGene mapping preview (first few rows):")
    print(preview_df(mapping_df))
    
    # 3. Apply the gene mapping to convert probe-level data to gene expression data
    print("\nApplying gene mapping to convert probe-level measurements to gene expression data...")
    gene_data = apply_gene_mapping(gene_data, mapping_df)
    
    # Check the resulting gene expression data
    print(f"Successfully created gene expression data with {len(gene_data.index)} genes and {gene_data.shape[1]} samples")
    print("\nPreview of mapped gene expression data (first few genes):")
    print(preview_df(gene_data.head()))
    
    # Create the output directory if it doesn't exist
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    
    # Save the gene expression data
    gene_data.to_csv(out_gene_data_file)
    print(f"Gene expression data saved to {out_gene_data_file}")
    
except Exception as e:
    print(f"Error during gene mapping: {e}")
    is_gene_available = False


Creating gene mapping dataframe...


Created mapping dataframe with 28566 entries

Gene mapping preview (first few rows):
{'ID': ['ILMN_35826', 'ILMN_25544', 'ILMN_10519', 'ILMN_17234', 'ILMN_19244'], 'Gene': ['LOC441782', 'JMJD1A', 'NCOA3', 'LOC389834', 'C17orf77']}

Applying gene mapping to convert probe-level measurements to gene expression data...
Successfully created gene expression data with 18401 genes and 117 samples

Preview of mapped gene expression data (first few genes):
{'GSM1054230': [0.90327597, 1.719921405, 27.15464165, 0.893225647, 1.357178552], 'GSM1054231': [0.821579705, 8.432092685, 17.07091413, 0.963601712, 1.423178661], 'GSM1054232': [0.865428376, 1.797851622, 34.4908899, 0.996713509, 1.518340034], 'GSM1054233': [0.890772454, 6.514503905, 9.773920808, 1.108153429, 1.352149752], 'GSM1054234': [0.986433802, 1.802559848, 31.28273099, 0.855068422, 1.415984108], 'GSM1054235': [1.197420108, 1.728039444, 20.56960997, 1.076571746, 1.592974841], 'GSM1054236': [0.886271319, 1.944330061, 65.49547785, 0.94327142

Gene expression data saved to ../../output/preprocess/Kidney_Papillary_Cell_Carcinoma/gene_data/GSE42977.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print("\nNormalizing gene symbols...")
try:
    # Load gene data that was saved in step 6
    gene_data = pd.read_csv(out_gene_data_file, index_col=0)
    print(f"Loaded gene data with {gene_data.shape[0]} genes and {gene_data.shape[1]} samples")
    
    # Normalize gene symbols using NCBI Gene database
    gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"After normalization: {gene_data.shape[0]} genes")
    
    # Save the normalized gene data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
    
    is_gene_available = True
except Exception as e:
    print(f"Error normalizing gene data: {e}")
    is_gene_available = False

# 2. Handle clinical data - based on step 2 determination that trait is not available
print("\nChecking clinical data availability...")
# From step 2: "The trait we are looking for (Kidney_Papillary_Cell_Carcinoma) is not present"
is_trait_available = False
note = "Dataset does not contain kidney papillary cell carcinoma samples. It focuses on Malignant Pleural Mesothelioma (MPM)."

# 3. Validate and save the cohort information
print("\nPerforming final validation...")
is_biased = True  # Not applicable since trait is unavailable
linked_data = pd.DataFrame()  # Empty DataFrame since we can't link without trait data

is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

print(f"Dataset usability for {trait} association studies: {is_usable}")
if not is_usable:
    print(f"Reason: {note}")


Normalizing gene symbols...
Loaded gene data with 18401 genes and 117 samples


After normalization: 17207 genes


Normalized gene data saved to ../../output/preprocess/Kidney_Papillary_Cell_Carcinoma/gene_data/GSE42977.csv

Checking clinical data availability...

Performing final validation...
Abnormality detected in the cohort: GSE42977. Preprocessing failed.
Dataset usability for Kidney_Papillary_Cell_Carcinoma association studies: False
Reason: Dataset does not contain kidney papillary cell carcinoma samples. It focuses on Malignant Pleural Mesothelioma (MPM).
